In [2]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp

import random
from itertools import chain, combinations, tee
import time

# Sample CVXPY code:

In [ ]:
x = cp.Variable(2)
y = cp.Variable(2)
v_fixed = np.array([0, 1])
objective = cp.Minimize(cp.sum_squares(x - y) + cp.sum_squares(x - v_fixed))
constraints = []
# for i in range(2):
#     constraints += [x[i] >= 2]
# constraints += [x[i] >=2 for i in range(2)]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print("x.value:", x.value)
print("y.value:", y.value)
print()



In [ ]:
# Problem data.
m = 30
n = 20
np.random.seed(1)
A = np.random.randn(m, n)
b = np.random.randn(m)

# Construct the problem.
x = cp.Variable(n)
objective = cp.Minimize(cp.sum_squares(A @ x - b))
constraints = [0 <= x, x <= 1]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print("x.value:", x.value)
print()
# The optimal Lagrange multiplier for a constraint is stored in
# `constraint.dual_value`.
print("constraints[0].dual_value:", constraints[0].dual_value)

# LP, Comparison between CVXPY and Gurobi:

In [135]:
c_1 = np.array([-1, 1])
c_2 = np.array([-1, -1])

In [153]:
x_cvxpy = cp.Variable(2)
prob = cp.Problem(cp.Minimize(x_cvxpy[0]), [c_1 @ x_cvxpy <= -3, c_2 @ x_cvxpy <= -3])
prob.solve()
x_cvxpy.value

array([ 3.00000000e+00, -5.09761577e-17])

In [151]:
m_QP = gp.Model()
x_gurobi = m_QP.addMVar(shape = 2, name = 'x_gurobi')
m_QP.addConstr((- x_gurobi[0] + x_gurobi[1] <= -3 ))
m_QP.addConstr((- x_gurobi[0] - x_gurobi[1] <= -3 ))

m_QP.setObjective(x_gurobi[0], GRB.MINIMIZE)
m_QP.update()
m_QP.optimize()
print()
print(x_gurobi.x)


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x6534bd8a
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 3e+00]
Presolve removed 2 rows and 2 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.000000000e+00

[3. 0.]


# QP, Comparison between CVXPY and Gurobi:

In [3]:
# Generate a random non-trivial quadratic program.
m = 15
n = 10
p = 5
np.random.seed(1)
P_gen = np.random.randn(n, n)
P = P_gen.T @ P_gen + np.eye(n)
q = np.random.randn(n)
G = np.random.randn(m, n)
h = G @ np.random.randn(n)
A = np.random.randn(p, n)
b = np.random.randn(p)

In [4]:
# P = np.array([[2, 1], [1, 2]])
# q = np.array([3, 4])
# n = 2

In [5]:
# Via CVXPY:

# Define and solve the CVXPY problem.
x_cvxpy = cp.Variable(n)
prob = cp.Problem(cp.Minimize((1/2)*cp.quad_form(x_cvxpy, P) + q.T @ x_cvxpy),
                 [G @ x_cvxpy <= h,
                  A @ x_cvxpy == b])
# prob = cp.Problem(cp.Minimize((1/2)*cp.quad_form(x_cvxpy, P) + q.T @ x_cvxpy))
prob.solve()

# Print result.
print("\nThe optimal value is", prob.value)
print("A solution x is")
print(x_cvxpy.value)
# print("A dual solution corresponding to the inequality constraints is")
# print(prob.constraints[0].dual_value)


The optimal value is 110.89146278608585
A solution x is
[-1.75612738  0.09726873 -1.57082808 -2.10310111  1.27660026  0.60664563
 -3.22096183  3.30465178  0.55753984 -1.92167322]


In [6]:
# Via Gurobi:

# Initialize model
m_QP = gp.Model()
# Add variables to model
x_gurobi = m_QP.addMVar((n), name = "x", lb = -10000.0, ub = 10000.0)
m_QP.addConstr((G @ x_gurobi <= h))
m_QP.addConstr((A @ x_gurobi == b))

m_QP.setObjective((0.5* x_gurobi.T @ P @ x_gurobi) + (q.T @ x_gurobi), gp.GRB.MINIMIZE)

# m_QP.update()
m_QP.optimize()


Set parameter Username
Academic license - for non-commercial use only - expires 2025-02-03
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 20 rows, 10 columns and 200 nonzeros
Model fingerprint: 0x5fe21bfb
Model has 55 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e-03, 3e+00]
  Objective range  [3e-02, 1e+00]
  QObjective range [7e-02, 1e+01]
  Bounds range     [1e+04, 1e+04]
  RHS range        [1e-01, 7e+00]
Presolve time: 0.01s
Presolved: 20 rows, 10 columns, 200 nonzeros
Presolved model has 55 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 9
 AA' NZ     : 4.060e+02
 Factor NZ  : 4.350e+02
 Factor Ops : 8.555e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal     

In [7]:
np.array([m_QP.getVars()[i].X for i in range(n)])

array([-1.75612607,  0.09727235, -1.57084287, -2.10311373,  1.27659858,
        0.60663089, -3.22098518,  3.30466016,  0.5575215 , -1.92170692])

In [8]:
x_cvxpy.value

array([-1.75612738,  0.09726873, -1.57082808, -2.10310111,  1.27660026,
        0.60664563, -3.22096183,  3.30465178,  0.55753984, -1.92167322])

In [48]:
# np.linalg.solve(P, -q)

# Scratch Work:

In [ ]:
# 20*np.linspace(0, 1, 21)[0:]
# np.linspace(toll_arr[0], toll_arr[-3]*T,19)